In [ ]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
import nltk
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import backend as K
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Embedding, Input, RepeatVector
from keras.optimizers import SGD

: 

In [ ]:
true = pd.read_csv('True.csv')
fake = pd.read_csv('Fake.csv')
true['label'] = 1
fake['label'] = 0
frames = [true.loc[:10000][:], fake.loc[:10000][:]]  #Edit this to control the size of dataset
df = pd.concat(frames)
df.head()

: 

In [ ]:
X = df.drop('label', axis=1) 
y = df['label']
df = df.dropna()
df2 = df.copy()
df2 = df2.reset_index(drop = True)


: 

In [ ]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(df2)):
    review = re.sub('[^a-zA-Z]', ' ', df2['text'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

: 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v = TfidfVectorizer(max_features=5000, ngram_range=(1,3))
X = tfidf_v.fit_transform(corpus).toarray()
y = df2['label']

: 

In [ ]:
def baseline_model() : 
    model = Sequential()
    model.add(Dense(256, activation = 'relu', kernel_initializer = 'normal'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation = 'relu', kernel_initializer = 'normal'))
    model.add(Dropout(0.5))
    model.add(Dense(80, activation = 'relu', kernel_initializer = 'normal'))
    model.add(Dense(1, activation = 'sigmoid', kernel_initializer = 'normal'))
    
    sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    return model

    

: 

In [ ]:
model = baseline_model()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

model.fit(X_train, y_train, epochs=30, batch_size=32)
results = model.evaluate(X_test, y_test)

print(model.predict(X_test))
print(results)


: 

In [ ]:
import pickle 
from keras.models import load_model

# vectorname = 'vectorizer.pkl'
# pickle.dump(tfidf_v, open(vectorname, 'wb'))
# model.save('nn_model.keras')
model_json = model.to_json()
with open('model_json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")

: 

: 